In [12]:
import warnings
warnings.filterwarnings("ignore")
import numpy as np
import pandas as pd
import os
import re
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics import accuracy_score,f1_score,classification_report,precision_score, recall_score
from sklearn.manifold import TSNE
from sklearn.linear_model import LogisticRegression
from keras.utils import np_utils
import keras
import matplotlib.pyplot as plt
from keras.preprocessing import sequence, text
from keras.callbacks import EarlyStopping
from keras.layers import GlobalMaxPooling1D, Conv1D, MaxPooling1D, Flatten, Bidirectional, SpatialDropout1D
from keras.models import Sequential
from keras.layers.recurrent import LSTM, GRU
from keras.layers.core import Dense, Activation, Dropout
from keras.layers.embeddings import Embedding
from keras.layers.normalization import BatchNormalization
from keras.utils import np_utils
import keras_metrics
from plotly import tools
import plotly.offline as py
py.init_notebook_mode(connected=True)
import plotly.graph_objs as go
from sklearn.cluster import KMeans

In [2]:
#load the data
train = pd.read_csv("train.csv")
train.head()

,qid,question_text,target
0,00002165364db923c7e6,How did Quebec nationalists see their province...,0
1,000032939017120e6e44,"Do you have an adopted dog, how would you enco...",0
2,0000412ca6e4628ce2cf,Why does velocity affect time? Does velocity a...,0
3,000042bf85aa498cd78e,How did Otto von Guericke used the Magdeburg h...,0
4,0000455dfa3e01eae3af,Can I convert montra helicon D to a mountain b...,0


In [3]:
from sklearn.model_selection import train_test_split

train_df, test_df = train_test_split(train, test_size=0.20, random_state=42)

In [4]:
train_count = train.groupby('target').count()
train_count.qid[0] / (train_count.qid[0] + train_count.qid[1])

0

In [5]:
from sklearn.feature_extraction.text import TfidfVectorizer
vect_word = TfidfVectorizer(max_features=10000, lowercase=True, analyzer='word',stop_words= 'english',ngram_range=(1,2),dtype=np.float32)
train_original = train_df
test_original = test_df
train_df = train_original.sample(frac = 0.05)
test_df = test_original.sample(frac = 0.05)
train_vectors = vect_word.fit_transform(train_df['question_text'])
test_vect = vect_word.transform(test_df['question_text'])

In [13]:
kmeans = KMeans(n_clusters=2).fit(train_vectors)
preds = kmeans.predict(test_vect)
accuracy = accuracy_score(test_df['target'], preds)
print accuracy
precision = precision_score(test_df['target'], preds)
print 'precision', precision
recall = recall_score(test_df['target'], preds)
print 'recall', recall

0.8904371793890208
precision 0.012678288431061807
recall 0.00980392156862745


In [15]:
print classification_report(test_df['target'], preds)

              precision    recall  f1-score   support

           0       0.93      0.95      0.94     12245
           1       0.01      0.01      0.01       816

   micro avg       0.89      0.89      0.89     13061
   macro avg       0.47      0.48      0.48     13061
weighted avg       0.88      0.89      0.88     13061

